HTML / XML  =>  DOM (-> BeautifulSoup의 도움)
            => CSSOM (selector; pseudo)

selector (*)  { X }
tags, #id, .class, .class1.class2
[attr], [attr = value], [attr^=, $=]
( ) => 자손,  > => 자식, + => 내 다음 형제

:nth-of-type, :first/last-child, :nth-child, ...
:has, :not

=> Crawling/Scraping
while 더이상 url 없을 때까지
    : robots.txt 파싱
    : HTTP Req/Resp ===> Resp.body (Scraping) => content-type()
    : anchor(a;hyperlink) => URL (*; urljoin)
    : link => regularization (full url)
    : visited O/X => URL Pool => ()
    : pop(url) => (BFS/DFS/Focused => 탐색 전략)



HyperLinks = ?
[Tags]
A, FORM, IMG, VIDEO, AUDIO, META, SCRIPT, STYLES, IFRAME, ...
[Attributes]
HREF, SRC, ACTION
=> * [HREF|SRC|ACTION]

In [112]:
from requests import request

headers = {
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

In [3]:
url = 'https://search.naver.com/search.naver'

params = {
    'where' : 'nexearch', 
    'query' : '뉴진스'
}

resp = request('GET', url, params= params, headers = headers)

In [4]:
resp.status_code, resp.reason, resp.headers

(200,
 'OK',
 {'Date': 'Mon, 10 Jul 2023 01:31:41 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': 'page_uid=i6WQQdp0JXVssZKU41hssssstQ4-068911; path=/; domain=.naver.com, _naver_usersession_=h+W2EgQHCkZaYNjFdHpuGw==; path=/; expires=Mon, 10-Jul-23 01:36:41 GMT; domain=.naver.com, nx_ssl=2; Domain=.naver.com; Path=/; Expires=Wed, 09-Aug-2023 01:31:41 GMT;', 'X-Frame-Options': 'SAMEORIGIN', 'X-XSS-Protection': '1; report=/p/er/post/xss', 'Cache-Control': 'no-cache, no-store, must-revalidate, max-age=0', 'Pragma': 'no-cache', 'Referrer-Policy': 'unsafe-url', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Server': 'nxg', 'Accept-CH': 'Sec-CH-UA, Sec-CH-UA-Arch, Sec-CH-UA-Bitness, Sec-CH-UA-Full-Version-List, Sec-CH-UA-Mobile, Sec-CH-UA-Model, Sec-CH-UA-Platform, Sec-CH-UA-Platform-Version, Sec-CH-UA-WoW64'})

In [5]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(resp.text, 'html5lib')

In [9]:
len(dom.select('[href]')), len(dom.select('[src]')), len(dom.select('[action]'))
# 이렇게 다양한 태그들로 포함이 된다는 걸 확인할 수 있다. 

(760, 252, 3)

In [10]:
dom.select('[href]')[0], dom.select('[src]')[0], dom.select('[action]')[0]


(<link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico" rel="shortcut icon"/>,
 <script crossorigin="anonymous" src="https://ssl.pstatic.net/sstatic/sdyn.js?f=/au/pc/_nx/jindo_1.5.3_171116.js+/search/js/nhn.Component.js+/search/js/jindo.Component.1.1.0.js+/search/js/jindo.component.library_120927.js+/au/s/pc/_common/jindo/jindo.Rolling_140526.js+/fe/sfe/common/pc-1.2.3.js+/fe/sfe/nx_jquery/pc/nx_jquery_221215.js+/fe/meerkat/logger/sfe/naver.common.meerkat.logger.sfe_pc_230601.js+/fe/sfe/naver_autocomplete/pc/naver.common.atcmp.naver_web_230216.js+/fe/sfe/web-vitals/web-vitals_230627.js+/fe/sfe/scrollLog/Controller_220714.js&amp;o=search"></script>,
 <form action="?" id="nx_search_form" method="get" name="search" onsubmit="return nx_form_submit(this)" role="search"> <fieldset class="greenwindow"> <legend>검색</legend> <input name="sm" type="hidden" value="tab_hty.top"/> <input name="where" type="hidden" value="nexearch"/> <div class="greenbox"> <input accesskey=

In [11]:
dom.select('[href]')[0].attrs['href'], dom.select('[src]')[0].attrs['src'], dom.select('[action]')[0].attrs['action']
# 이 주소의 path가 명확하지 않음. -> dom.select('[action]')[0].attrs['action']
# 'https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico' -> 전혀 다른 형태의 도메인이 나오는데 이를 urljoin을 활용하여.

('https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico',
 'https://ssl.pstatic.net/sstatic/sdyn.js?f=/au/pc/_nx/jindo_1.5.3_171116.js+/search/js/nhn.Component.js+/search/js/jindo.Component.1.1.0.js+/search/js/jindo.component.library_120927.js+/au/s/pc/_common/jindo/jindo.Rolling_140526.js+/fe/sfe/common/pc-1.2.3.js+/fe/sfe/nx_jquery/pc/nx_jquery_221215.js+/fe/meerkat/logger/sfe/naver.common.meerkat.logger.sfe_pc_230601.js+/fe/sfe/naver_autocomplete/pc/naver.common.atcmp.naver_web_230216.js+/fe/sfe/web-vitals/web-vitals_230627.js+/fe/sfe/scrollLog/Controller_220714.js&o=search',
 '?')

In [16]:
from requests.compat import urlparse, urljoin

urlparse(resp.url).path, \
urljoin(resp.url, '?a=b'), \
urljoin(resp.url, 'http://www.test.com') # ? 뒤에 param이 있으므로 이를 바꾸겠다는 뜻이다.


('/search.naver',
 'https://search.naver.com/search.naver?a=b',
 'http://www.test.com')

In [18]:
dom.select('[action]')[0].attrs['method'] # get 메서드가 아닌 형태는 다르게 조정해야 할 수도 있음.

'get'

In [19]:
for node in dom.select('[action]')[0]. select('input'):
    print(node.attrs['name'] + '=' + node.attrs['value'])

# 파라미터 쌍이다. 경우에 따라 쓰일수도 있고, 안 쓰일수도 있지만, 지금은 안 쓰여 있다. 

sm=tab_hty.top
where=nexearch
query=뉴진스
oquery=뉴진스
tqi=i6WQQdp0JXVssZKU41hssssstQ4-068911
acq=
acr=
qdt=
acir=
os=
bid=
pkid=
eid=
mra=


In [21]:
from urllib.parse import parse_qs
parse_qs(urlparse(resp.url).query)
# link를 통해 정규화하는 과정 (전체 url) -> link는 naver.com 이라면 파라미터를 추가하는 과정이 regularization이라고 할 수 있다.

{'where': ['nexearch'], 'query': ['뉴진스']}

In [23]:
dom.select('[href^="javascript"]')
# onclick -> event trigger -> selenium 보충.

[<a class="mp3_down" href="javascript:;" nocr="" onclick="goOtherTCR(this, 'a=nco_xgh*a.adown&amp;r=1&amp;i=00000000_000001E62A69');window.open('https://vibe.naver.com/mp3/mp3CartRoute?trackIds=65992968', '', 'width=650,height=572,scrollbars=yes,resizable=yes');"> <span class="text">MP3</span> </a>,
 <a class="spnew_af item item_save _keep_save" data-cr-off="a=itb_bas*b.kepoff&amp;r=15&amp;i=SYS-0000000005344485.90000003_0000000000000033F48A0DE8&amp;g=%7B%22bid%22%3A%22SYS-0000000005344485%22%2C%22docRank%22%3A1%7D" data-cr-on="a=itb_bas*b.kepon&amp;r=15&amp;i=SYS-0000000005344485.90000003_0000000000000033F48A0DE8&amp;g=%7B%22bid%22%3A%22SYS-0000000005344485%22%2C%22docRank%22%3A1%7D" href="javascript:;" role="button">Keep에 저장</a>,
 <a class="spnew_af item item_save _keep_save" data-cr-off="a=itb_bas*b.kepoff&amp;r=16&amp;i=SYS-0000000005344485.90000003_0000000000000033F4709394&amp;g=%7B%22bid%22%3A%22SYS-0000000005344485%22%2C%22docRank%22%3A2%7D" data-cr-on="a=itb_bas*b.kepon&amp;r=1

In [24]:
dom.select('[href^="#"]')
# 영역 바로가기 역할이므로 걸러야 함. 어차피 같은 내용.

[<a href="#lnb"><span>메뉴 영역으로 바로가기</span></a>,
 <a href="#content"><span>본문 영역으로 바로가기</span></a>,
 <a aria-pressed="false" class="bt_setkr" href="#" id="ke_kbd_btn" onclick="return tCR('a=sch.ime');" role="button"><i class="spnew ico_keyboard">한글 입력기</i></a>,
 <a aria-pressed="false" class="bt_atcp _btn_arw" data-atcmp-element="" href="#" id="nautocomplete" role="button"><i class="spnew ico_arrow">자동완성 레이어</i></a>,
 <a aria-pressed="false" class="item _delAll" href="#" role="button">전체삭제</a>,
 <a class="kwd" href="#"> <span class="fix"><span class="common_ico_kwd"><i class="imsc ico_search"></i></span><span>@txt@</span></span> </a>,
 <a aria-pressed="false" class="bt_item _del" href="#" role="button"><i class="imsc ico_del">삭제</i></a>,
 <a class="close _keywordOnOff" href="#">자동저장 끄기</a>,
 <a class="link_item" href="#"> <span class="common_ico_kwd"><i class="imsc ico_search"></i></span> <div class="dsc_area"> <span class="tit">@5@회 로또당첨번호</span> <span class="dsc"> <span class="item">동행

            root ; WWW
        A           B

BFS (앞에서 ; queue) => 
=>  관련도 순으로 ... 

DFS (뒤에서 ; stack) => 너무 깊으면, 못 빠져나옴. 

Focused Crawling (전략) => depth 한계치를 정해둔다. (특정 도메인 등등)
                        => 웹툰, 뉴스 등 한계 

In [25]:
from urllib.robotparser import RobotFileParser
rb = RobotFileParser(주소)
rb.read()
rb.can_fetch(agent, path) # T/F

In [110]:
from requests.exceptions import HTTPError
import re

ROBOT = dict()

def canFetch(url, agent = '*', path = '/'):
    k = urlparse(url).netloc

    if k not in ROBOT:
        robot = urljoin(url, '/robots.txt') # robot이 없으면, 3.1
        resp = request('GET', robot)

        try:
            resp.raise_for_status()
        except HTTPError as e:
            print(e) 

        if resp.status_code == 200:
            ua = '*'
            ROBOT[k] = list()

            for l in resp.text.lower().splitlines():
                a = re.search(r'user-agent:(.+)', l, re.IGNORECASE) # parsing -> user-agent /Allow/Disallow
                if a:
                    ua = a.group(1).strip()
                else:
                    p = re.search(r'((?:dis)?allow):(.+)', l, re.IGNORECASE) # User-agent 물어보고, 지금 작업 중인 ua를 ua로 설정

                    if p:
                        allow = False if p.group(1) == 'disallow' else True
                        ROBOT[k].append((ua, p.group(2).strip(), allow)) # allow/disallow를 key = tuple(ua, path, allow/disallow)

            # rp = RobotFileParser(robot)
            # rp.read()
            # rp.can_fetch(agent, path)
    
    if k in ROBOT: # 로봇이 있으면.
        candidates = list()
        
        for l in ROBOT[k]:
            if l[0] == agent and re.match(l[1].replace('*', '[^/]+?'), path): # param.agent 값으로 path에 re
                candidates.append(l)
        #candidates = list(filter(lambda l:l[0] == agent and re.match(l[1].replace('*', '[^/]+?', path))))

        candidates = sorted(candidates, key=lambda c: len(c[1]), reverse=True)
        
        if len(candidates) > 0:
            return candidates[0][-1]
        return True


    return True



In [64]:
canFetch('https://www.google.com', agent= '*', path='/search/?adksmfd')

False

In [116]:
urls = ['https://www.google.com/search?q=%EB%89%B4%EC%A7%84%EC%8A%A4&source=hp&ei=W3WrZLuILIbY1e8PqPit6As&iflsig=AD69kcEAAAAAZKuDa38MQ51JwvPHwYd83AuvhAqMfll5&ved=0ahUKEwj7ofv7k4OAAxUGbPUHHSh8C70Q4dUDCAs&uact=5&oq=%EB%89%B4%EC%A7%84%EC%8A%A4&gs_lcp=Cgdnd3Mtd2l6EAMyCwguEIAEELEDEIMBMgQIABADMgsILhCABBCxAxCDATILCC4QgAQQsQMQgwEyCwguEIAEELEDEIMBMggIABCABBCxAzILCAAQgAQQsQMQgwEyCwgAEIAEELEDEIMBMggILhCABBCxAzILCAAQgAQQsQMQgwE6DgguEIAEELEDEMcBENEDOgUIABCABFDUAVixB2ChCGgCcAB4AYABe4gBwAWSAQMwLjaYAQCgAQGwAQA&sclient=gws-wiz']
visited = []
i = 0
while urls: 
    #  limitation
    if i > 10:
        break
    i += 1

    url = urls.pop(0)
    visited = []
    try:
        print(urlparse(url).path, canFetch(url, path = urlparse(url).path))
    except:
        print(url)
    resp = request('GET', url, headers=headers)
    # <meta> index/noindex, follow/nofollow -> 하지만 이를 다 확인하면 가져올 수 있는게 없다.
    # meta = dom.select_one('meta[name="robots"]')
    # meta.attrs['content'].split(',')
    if resp.status_code == 200:
        visited.append(url)
        if re.search('xml|html', resp.headers['content-type']):
            dom = BeautifulSoup(resp.text, 'lxml')
            a = list()
            for node in dom.select('[href], [src]'):
                temp = urljoin(url, node.attrs['href' if node.has_attr('href') else 'src']) # , = '또는'
                
                if not re.match(r'#|javascript|data:', temp):
                    newurl = urljoin(url, temp)
                    a.append(newurl)
#                else:
#                    print(temp, urlparse(urljoin(url, temp)))
            print(len(a))
            urls += list(filter(lambda link:link not in visited and link not in urls, a))
    else:
        if resp.status_code >= 500:
            urls.append(url) # 나중에 다시 검토.
        else:
            visited.append(url)

            # 방문 이력, url pool 관리
        # else 일 때, 처리 X, 500 => 
    # 서버 부하 delay 

/search False
96
/search False
128
/websearch/answer/181196 True
40
/webhp False
20
/images/branding/googlelogo/2x/googlelogo_color_92x30dp.png False
/tia/tia.png False
/search False
98
/search False
52
/search False
35
/search False
71
/w/NewJeans True
1088


In [84]:
a[100]

'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='

In [104]:
urlparse('https://search.naver.com/path/#lnb'), \
urlparse('https://search.naver.com/path?a=b')
# fragment가 달라도 된다. -> 같은 내용의 페이지에서 위치만 다른 내용이다. 

(ParseResult(scheme='https', netloc='search.naver.com', path='/path/', params='', query='', fragment='lnb'),
 ParseResult(scheme='https', netloc='search.naver.com', path='/path', params='', query='a=b', fragment=''))